## Import required Python libraries

In [ ]:
# import required libraries
import pandas as pd
import sqlite3

In [ ]:
# %pip install ipython-sql
# no need to install again once previously installed

In [ ]:
# load the sql module to iPython
%load_ext sql

## Connect with Database

In [ ]:
# 'Cell Magic'
%sql sqlite:///MyPizza.db

In [ ]:
# begin test with sql query to list all the columns with limit to fetch first 5 rows.

In [ ]:
%%sql
select * from pizzas
limit 5

create a sample table

In [ ]:
%%sql
create table sample_table (
pizza_id int not NULL
)

Drop a table

In [ ]:
%%sql
drop table sample_table

List all tables in a Database

In [ ]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';

## Basic SQL:

1. Retrieve the total number of orders placed.

In [ ]:
%%sql
select count(order_id) as total_orders
from orders

2. Calculate the total revenue generated from pizza sales.

In [ ]:
%%sql

SELECT ROUND(SUM(od.quantity * pz.price), 2) AS total_revenue
FROM   order_details AS od
       JOIN pizzas AS pz
         ON od.pizza_id = pz.pizza_id 

3. Identify the highest-priced pizza.

In [ ]:
%%sql
select 
  name, 
  max(price) 
from 
  pizzas as pz 
  join pizza_types as pt on pz.pizza_type_id = pt.pizza_type_id

4. Identify the most common pizza size ordered.

In [ ]:
%%sql

select 
  pz.size, 
  count(od.order_details_id) as common_size_ordered 
from 
  pizzas as pz 
  join order_details as od on pz.pizza_id = od.pizza_id 
group by 
  pz.size 
order by 
  common_size_ordered desc

5. List the top 5 most ordered pizza types along with their quantities.

In [ ]:
%%sql

SELECT 
  pizza_types.name, 
  sum(order_details.quantity) as order_quantity_sum 
from 
  pizzas 
  join order_details on pizzas.pizza_id = order_details.pizza_id 
  join pizza_types on pizzas.pizza_type_id = pizza_types.pizza_type_id 
GROUP by 
  pizza_types.name 
ORDER by 
  order_quantity_sum DESC 
limit 
  5

Note: The GROUP BY clause in SQL is mandatory when you are using aggregate functions (like COUNT(), SUM(), AVG(), MAX(), MIN(), etc.) alongside non-aggregated columns in the SELECT statement.

## Intermediate SQL:

1. Join the necessary tables to find the total quantity of each pizza category ordered.

In [ ]:
%%sql

SELECT 
  pizza_types.category, 
  sum(order_details.quantity) as total_category_orders 
from 
  pizzas 
  join order_details on pizzas.pizza_id = order_details.pizza_id 
  join pizza_types on pizzas.pizza_type_id = pizza_types.pizza_type_id 
GROUP by 
  pizza_types.category 
ORDER by 
  total_category_orders DESC

2. Determine the distribution of orders by hour of the day.

In [ ]:
%%sql
SELECT strftime('%H', order_time) AS hour, COUNT(order_id) as order_count
FROM orders
GROUP BY hour;

3. Join relevant tables to find the category-wise distribution of pizzas.

In [ ]:
%%sql
select 
  category, 
  count(pizza_type_id) as category_count 
from 
  pizza_types 
GROUP by 
  category
order by
  category_count DESC

Observed there is some incorrect rows present in pizza_types. Using DELETE query to remove non-relevent rows.

In [ ]:
%%sql
DELETE from pizza_types
WHERE category in ('L', 'M', 'S', 'XL', 'XXL')

In [ ]:
%%sql
select category, count(pizza_type_id) as category_count from pizza_types
GROUP by category

4. Group the orders by date and calculate the average number of pizzas ordered per day.

In [ ]:
%%sql

select 
  round(
    avg(sum_orders), 
    0
  ) as average_order_per_day 
FROM 
  (
    select 
      orders.order_date, 
      sum(order_details.quantity) as sum_orders 
    from 
      orders 
      join order_details on orders.order_id = order_details.order_id 
    GROUP by 
      orders.order_date
  )

5. Determine the top 3 most ordered pizza types based on revenue.

In [ ]:
%%sql

SELECT 
  pizza_types.name as pizza_name, 
  sum(
    order_details.quantity * pizzas.price
  ) as revenue 
from 
  order_details 
  JOIN pizzas on pizzas.pizza_id = order_details.pizza_id 
  JOIN pizza_types on pizza_types.pizza_type_id = pizzas.pizza_type_id 
GROUP by 
  pizza_name 
order by 
  revenue DESC 
limit 
  3

## Advanced SQL:

1. Calculate the percentage contribution of each pizza type to total revenue.

In [ ]:
%%sql

select 
  pizza_types.category, 
  round(
    sum(
      order_details.quantity * pizzas.price
    ) / (
      SELECT 
        round(
          sum(
            order_details.quantity * pizzas.price
          ), 
          2
        ) as total_sales 
      FROM 
        order_details 
        JOIN pizzas on pizzas.pizza_id = order_details.pizza_id
    ) * 100, 
    2
  ) as percent_revenue 
from 
  pizza_types 
  join pizzas on pizza_types.pizza_type_id = pizzas.pizza_type_id 
  join order_details on order_details.pizza_id = pizzas.pizza_id 
GROUP by 
  pizza_types.category 
order by 
  percent_revenue desc

2. Analyze the cumulative revenue generated over time.

3. Determine the top 3 most ordered pizza types based on revenue for each pizza category.

Learning source:
https://www.youtube.com/watch?v=zZpMvAedh_E&t=2773s